In [1]:
import os
import pandas as pd
from pptx import Presentation
import pdf2image
import win32com.client
import win32com.client.connect
from transformers import ViTFeatureExtractor
#from pptxtopdf import convert
#from datasets import Dataset
from PIL import Image, UnidentifiedImageError
import unicodedata
import numpy as np
from transformers import AutoTokenizer, AutoProcessor, TrainingArguments, BitsAndBytesConfig
from trl import SFTTrainer
from peft import LoraConfig
from transformers import Qwen2VLForConditionalGeneration
from qwen_vl_utils import process_vision_info
import torch

#poppler library path
os.environ["PATH"] = r"C:\Users\filip\OneDrive\Área de Trabalho\Cast Group\Projects\for_all_see_it_cast\poppler-24.07.0\Library\bin"

c:\Users\filip\OneDrive\Área de Trabalho\Cast Group\Projects\for_all_see_it_cast\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#pip install torch==2.5.1+cu121 torchaudio==2.5.1+cu121 torchvideo==0.20.1+cu121 --index-url https://download.pytorch.org/whl/cu121


In [3]:
#pip install trl peft qwen_vl_utils 

In [ ]:
import os
import win32com.client
import pandas as pd

def extrair_slides_e_comentarios(diretorio):
    # Inicializa o PowerPoint
    ppt_app = win32com.client.Dispatch("PowerPoint.Application")
    ppt_app.Visible = True
        
    # Lista para armazenar os dados
    dados = []

    # Itera sobre todos os arquivos no diretório
    for arquivo in os.listdir(diretorio):
        if arquivo.endswith('.pptx'):
            pptx_path = os.path.join(diretorio, arquivo)
            apresentacao = ppt_app.Presentations.Open(pptx_path)

            # Processa cada slide
            for slide_index, ppt_slide in enumerate(apresentacao.Slides):
                # Define o caminho da imagem
                imagem_path = os.path.join(diretorio, f"{arquivo[:-5]}_slide_{slide_index + 1}.png")
                
                # Salva o slide como imagem
                ppt_slide.Export(imagem_path, 'PNG')
                
                # Extrai comentários
                comentarios = [comment.Text for comment in ppt_slide.Comments]

                # Adiciona os dados ao DataFrame
                for comentario in comentarios:
                    dados.append({
                        'nome_arquivo': arquivo,
                        'image': imagem_path.replace('\\','/'),
                        'pergunta': 'O que tem nessa imagem?',
                        'resposta': comentario.replace('\n\n',' ').replace('\n',' ').strip()
                    })

            # Fecha a apresentação
            apresentacao.Close()

    # Cria um DataFrame a partir dos dados
    df = pd.DataFrame(dados)

    # Encerra o PowerPoint
    ppt_app.Quit()
    
    return df


In [ ]:

# Exemplo de uso
diretorio_pptx = "C:\\Users\\filip\\OneDrive\\Área de Trabalho\\Cast Group\\Projects\\for_all_see_it_cast\\data\\pptx\\"
df_resultado = extrair_slides_e_comentarios(diretorio_pptx)

In [3]:
df_resultado.head()

,nome_arquivo,image,pergunta,resposta
0,Decima_Jornada_da_Agilidade_Cast.pptx,C:/Users/filip/OneDrive/Área de Trabalho/Cast ...,O que tem nessa imagem?,#PraTodosVerem Fundo: A imagem tem um fundo pr...
1,Decima_Jornada_da_Agilidade_Cast.pptx,C:/Users/filip/OneDrive/Área de Trabalho/Cast ...,O que tem nessa imagem?,#PraTodosVerem Fundo: O fundo da imagem é bran...
2,Decima_Jornada_da_Agilidade_Cast.pptx,C:/Users/filip/OneDrive/Área de Trabalho/Cast ...,O que tem nessa imagem?,#PraTodosVerem Fundo: O fundo da imagem é bran...
3,Decima_Jornada_da_Agilidade_Cast.pptx,C:/Users/filip/OneDrive/Área de Trabalho/Cast ...,O que tem nessa imagem?,#PraTodosVerem Fundo: O fundo da imagem é divi...
4,Decima_Jornada_da_Agilidade_Cast.pptx,C:/Users/filip/OneDrive/Área de Trabalho/Cast ...,O que tem nessa imagem?,#PraTodosVerem Fundo: O fundo da imagem é divi...


In [2]:

# processor = LlavaNextProcessor.from_pretrained("llava-hf/llava-v1.6-mistral-7b-hf")
# model = LlavaNextForConditionalGeneration.from_pretrained("llava-hf/llava-v1.6-mistral-7b-hf", torch_dtype=torch.float16, low_cpu_mem_usage=True)
# model.to("cuda:0")  # Move model to GPU if available

model_id = "Qwen/Qwen2-VL-2B-Instruct"

# quantization_config = BitsAndBytesConfig(
#     load_in_4bit=True,
# )

In [6]:
# model = Qwen2VLForConditionalGeneration.from_pretrained(model_id,
#                                                       quantization_config=quantization_config,
#                                                       torch_dtype=torch.float16)

In [7]:
# note the image is not provided in the prompt its included as part of the "processor"
prompt= """Create a detail description based on the provided ##PERGUNTA## and image. 
Only return description. The description should be SEO optimized and for a better mobile search experience.
 
##PERGUNTA##: {pergunta}
"""
 
system_message = "You are an expert image description writer for blind person."

In [8]:
processor = AutoProcessor.from_pretrained(model_id)

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
# tokenizer.chat_template = LLAVA_CHAT_TEMPLATE

processor.tokenizer = tokenizer
feature_extractor = ViTFeatureExtractor.from_pretrained(model_id)

C:\Users\filip\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\models\vit\feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [ ]:

# Função para remover acentuação
def remove_accentuation(text):
    normalized_text = unicodedata.normalize('NFKD', text)
    return ''.join(char for char in normalized_text if not unicodedata.combining(char))



# Convert dataset to OAI messages       
def format_data(sample):


    image = Image.open(sample.image).convert("RGB")
    width, height = image.size
    image = image.resize((int(0.4*width), int(0.4*height)))
    
    return {"messages": [
                {
                    "role": "system",
                    "content": [{"type": "text", "text": system_message}],
                },
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": prompt.format(pergunta=sample.pergunta)
                        },{
                            "type": "image",
                            "image": image,
                        }
                    ],
                },
                {
                    "role": "assistant",
                    "content": [{"type": "text", "text": sample.resposta}],
                },
            ],
        }
 
# Convert dataset to OAI messages
# need to use list comprehension to keep Pil.Image type, .mape convert image to bytes
dataset = [format_data(sample) for sample in df_resultado.itertuples()]
 
print(dataset[2]["messages"])


[{'role': 'system', 'content': [{'type': 'text', 'text': 'You are an expert image description writer for blind person.'}]}, {'role': 'user', 'content': [{'type': 'text', 'text': 'Create a detail description based on the provided ##PERGUNTA## and image. \nOnly return description. The description should be SEO optimized and for a better mobile search experience.\n \n##PERGUNTA##: O que tem nessa imagem?\n'}, {'type': 'image', 'image': <PIL.Image.Image image mode=RGB size=512x288 at 0x1EFCFD52850>}]}, {'role': 'assistant', 'content': [{'type': 'text', 'text': '#PraTodosVerem Fundo: O fundo da imagem é branco, com duas faixas curvas em vermelho no canto superior direito. No rodapé à esquerda, há uma barra horizontal com uma linha azul escura e uma azul clara, que se conectam à direita. Título: O título "Visão do Produto" está centralizado no topo da imagem, em fonte sans-serif preta e negrito. Texto: À esquerda, há um gráfico com cinco círculos concêntricos em tons de azul, cada um contend

In [11]:
import torch
from transformers import AutoModelForVision2Seq, AutoProcessor, BitsAndBytesConfig
 
 
# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)
 
# Load model and tokenizer
model = AutoModelForVision2Seq.from_pretrained(
    model_id,
    device_map="auto",
    # attn_implementation="flash_attention_2", # not supported for training
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config
)
processor = AutoProcessor.from_pretrained(model_id)

`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46
Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.80s/it]


In [13]:
# Preparation for inference
text = processor.apply_chat_template(
    dataset[2]["messages"], tokenize=False, add_generation_prompt=False
)
print(text)

<|im_start|>system
You are an expert image description writer for blind person.<|im_end|>
<|im_start|>user
Create a detail description based on the provided ##PERGUNTA## and image. 
Only return description. The description should be SEO optimized and for a better mobile search experience.
 
##PERGUNTA##: O que tem nessa imagem?
<|vision_start|><|image_pad|><|vision_end|><|im_end|>
<|im_start|>assistant
#PraTodosVerem Fundo: O fundo da imagem é branco, com duas faixas curvas em vermelho no canto superior direito. No rodapé à esquerda, há uma barra horizontal com uma linha azul escura e uma azul clara, que se conectam à direita. Título: O título "Visão do Produto" está centralizado no topo da imagem, em fonte sans-serif preta e negrito. Texto: À esquerda, há um gráfico com cinco círculos concêntricos em tons de azul, cada um contendo um texto que descreve níveis estratégicos do produto: No círculo maior e mais externo, está escrito "Business Strategy". No segundo círculo, "Product Vision

In [14]:
# example = dataset[2]
# image = example["image"]
# text = example["messages"]
# # resize image for smaller displaying
# width, height = image.size
# image = image.resize((int(0.3*width), int(0.3*height)))
# image

In [14]:
from peft import LoraConfig
 
# LoRA config based on QLoRA paper & Sebastian Raschka experiment
peft_config = LoraConfig(
        lora_alpha=16,
        lora_dropout=0.05,
        r=8,
        bias="none",
        target_modules=["v_proj"],
        task_type="CAUSAL_LM", 
)

In [15]:
from trl import SFTConfig
from transformers import Qwen2VLProcessor
from qwen_vl_utils import process_vision_info
 
args = SFTConfig(
    output_dir="C:\\Users\\filip\\OneDrive\\Área de Trabalho\\Cast Group\\Projects\\for_all_see_it_cast\\data\\model\\qwen2-2b-cast", # directory to save and repository id
    num_train_epochs=1,                     # number of training epochs
    per_device_train_batch_size=2,          # batch size per device during training
    gradient_accumulation_steps=8,          # number of steps before performing a backward/update pass
    gradient_checkpointing=True,            # use gradient checkpointing to save memory
    optim="adamw_torch_fused",              # use fused adamw optimizer
    #logging_steps=5,                       # log every 10 steps
    save_strategy="epoch",                  # save checkpoint every epoch
    learning_rate=2e-4,                     # learning rate, based on QLoRA paper
    bf16=True,                              # use bfloat16 precision
    tf32=True,                              # use tf32 precision
    max_grad_norm=0.3,                      # max gradient norm based on QLoRA paper
    warmup_ratio=0.03,                      # warmup ratio based on QLoRA paper
    lr_scheduler_type="constant",           # use constant learning rate scheduler
    push_to_hub=False,                       # push model to hub
    #report_to="tensorboard",                # report metrics to tensorboard
    gradient_checkpointing_kwargs = {"use_reentrant": False}, # use reentrant checkpointing
    dataset_text_field="", # need a dummy field for collator
    dataset_kwargs = {"skip_prepare_dataset": True} # important for collator
)
args.remove_unused_columns=False
 
# Create a data collator to encode text and image pairs
def collate_fn(examples):
    # Get the texts and images, and apply the chat template
    texts = [processor.apply_chat_template(example["messages"], tokenize=False) for example in examples]
    image_inputs = [process_vision_info(example["messages"])[0] for example in examples]
 
    # Tokenize the texts and process the images
    batch = processor(text=texts, images=image_inputs, return_tensors="pt", padding=True)
 
    # The labels are the input_ids, and we mask the padding tokens in the loss computation
    labels = batch["input_ids"].clone()
    labels[labels == processor.tokenizer.pad_token_id] = -100  #
    # Ignore the image token index in the loss computation (model specific)
    if isinstance(processor, Qwen2VLProcessor):
        image_tokens = [151652,151653,151655]
    else: 
        image_tokens = [processor.tokenizer.convert_tokens_to_ids(processor.image_token)]
    for image_token_id in image_tokens:
        labels[labels == image_token_id] = -100
    batch["labels"] = labels
 
    return batch

In [16]:
from trl import SFTTrainer
 
trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=dataset,
    data_collator=collate_fn,
    dataset_text_field="messages", # needs dummy value
    peft_config=peft_config,
    tokenizer=processor.tokenizer,
)

C:\Users\filip\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\huggingface_hub\utils\_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
C:\Users\filip\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\trl\trainer\sft_trainer.py:292: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
C:\Users\filip\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\trl\trainer\sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the valu

In [17]:
dataset

[{'messages': [{'role': 'system',
    'content': [{'type': 'text',
      'text': 'You are an expert image description writer for blind person.'}]},
   {'role': 'user',
    'content': [{'type': 'text',
      'text': 'Create a detail description based on the provided ##PERGUNTA## and image. \nOnly return description. The description should be SEO optimized and for a better mobile search experience.\n \n##PERGUNTA##: O que tem nessa imagem?\n'},
     {'type': 'image',
      'image': <PIL.Image.Image image mode=RGB size=512x288>}]},
   {'role': 'assistant',
    'content': [{'type': 'text',
      'text': '#PraTodosVerem Fundo: A imagem tem um fundo predominantemente branco à esquerda, com uma linha azul clara horizontal separando o título do restante do conteúdo. À direita, há um padrão visual composto por formas geométricas azuis em tons claros, representando uma rede conectada de pontos brilhantes interligados por linhas, sugerindo uma metáfora de conexão ou rede digital. Essa área direit

In [18]:
# start training, the model will be automatically saved to the hub and the o utput directory
trainer.train()

100%|██████████| 3/3 [00:23<00:00,  7.76s/it]

{'train_runtime': 23.2769, 'train_samples_per_second': 2.234, 'train_steps_per_second': 0.129, 'train_loss': 19.717657725016277, 'epoch': 0.92}


TrainOutput(global_step=3, training_loss=19.717657725016277, metrics={'train_runtime': 23.2769, 'train_samples_per_second': 2.234, 'train_steps_per_second': 0.129, 'total_flos': 408963619104768.0, 'train_loss': 19.717657725016277, 'epoch': 0.9230769230769231})

In [19]:

# save model 
trainer.save_model(args.output_dir)

Consuming VLM Model

In [3]:
# free the memory again
del model
del trainer
torch.cuda.empty_cache()

NameError: name 'model' is not defined

In [ ]:
import torch
from transformers import AutoProcessor, AutoModelForVision2Seq
 
#adapter_path = "./qwen2-2b-cast"

adapter_path = 'C:\\Users\\filip\\OneDrive\\Área de Trabalho\\Cast Group\\Projects\\for_all_see_it_cast\\data\\model\\qwen2-2b-cast'
processor_path = 'C:\\Users\\filip\\OneDrive\\Área de Trabalho\\Cast Group\\Projects\\for_all_see_it_cast\\data\\processor\\'

# Load Model base model
model = AutoModelForVision2Seq.from_pretrained(
  adapter_path,
  device_map="auto",
  torch_dtype=torch.float16
)
processor = AutoProcessor.from_pretrained(processor_path)

Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.05it/s]


In [20]:
import textwrap
def wrap_text(input_text, line_length):
    # Using textwrap to wrap the text
    wrapped_text = textwrap.fill(input_text, width=line_length)
    return wrapped_text

In [21]:
def extrair_slides_e_comentarios(diretorio):
    # Inicializa o PowerPoint
    ppt_app = win32com.client.Dispatch("PowerPoint.Application")
    ppt_app.Visible = True
        
    # Lista para armazenar os dados
    dados = []

    # Itera sobre todos os arquivos no diretório
    for arquivo in os.listdir(diretorio):
        if arquivo.endswith('.pptx'):
            pptx_path = os.path.join(diretorio, arquivo)
            apresentacao = ppt_app.Presentations.Open(pptx_path)

            # Processa cada slide
            for slide_index, ppt_slide in enumerate(apresentacao.Slides):
                # Define o caminho da imagem
                imagem_path = os.path.join(diretorio, f"{arquivo[:-5]}_slide_{slide_index + 1}.png")
                
                # Salva o slide como imagem
                ppt_slide.Export(imagem_path, 'PNG')

                
                dados.append({
                        'nome_arquivo': arquivo,
                        'image': imagem_path.replace('\\','/'),
                        'pergunta': 'O que tem nessa imagem?'
                    })

    # Fecha a apresentação
    apresentacao.Close()

    # Cria um DataFrame a partir dos dados
    df = pd.DataFrame(dados)

    # Encerra o PowerPoint
    ppt_app.Quit()

    return df

In [22]:
to_score = 'C:\\Users\\filip\\OneDrive\\Área de Trabalho\\Cast Group\\Projects\\for_all_see_it_cast\\data\\to_score'
df_score = extrair_slides_e_comentarios(to_score)

In [25]:
def invoke_output(img_path='data\\pptx\\Jornada da Agilidade Cast - com PTV 1_slide_23.png'):

    image = Image.open(img_path).convert("RGB")
    width, height = image.size
    image = image.resize((int(0.4*width), int(0.4*height)))

    conversation = [
    {
        "role": "user",
        "content": [
            {"type": "image"},
            {"type": "text", "text": "Descreva essa imagem, o que tem de informação nela? Nao repita o texto."},
        ],
    },
]
    prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)
    inputs = processor(image, prompt, return_tensors="pt").to("cuda:0")

    output = model.generate(**inputs, max_new_tokens=300)
    line_length = 100
    wrapped = wrap_text(processor.decode(output[0], skip_special_tokens=True).split('assistant')[2], line_length)

    return '#PraTodosVerem ' + wrapped


In [9]:
pptv = []
for img in df_score.image:
    text = invoke_output(img_path=img).replace('\n',' ')
    pptv.append(text)

ValueError: text input must be of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

In [56]:
df_score['PPTV'] = [x.replace('uma frase em português que diz:','uma frase que diz:') for x in pptv]

In [57]:
df_score.PPTV.values

array(['#PraTodosVerem  A imagem apresenta o logotipo da Cast Group, uma empresa brasileira especializada em soluções ágeis para a transformação digital. O texto na imagem diz: "Soluções Ágeis Acelerando a Transformação Digital". A imagem também inclui o endereço web: www.castgroup.com.br.',
       '#PraTodosVerem  A imagem mostra o logotipo da empresa Cast Group, que inclui uma imagem de um prédio de escritórios e um laptop com a marca Cast. A frase "Sobre nós" está em destaque, seguida de uma lista de informações: + 34 anos de experiência, + 10 mil projetos realizados, + 450 clientes ativos fazendo parte dessa história.',
       '#PraTodosVerem  A imagem apresenta uma apresentação de diferencial de uma empresa chamada Cast Group. As informações destacadas são:  1. Tecnologias inovadoras: A empresa está sempre em busca de soluções inovadoras para atender aos seus clientes e contatos. Ela é atualizada com avanços tecnológicos e novidades do mercado.  2. Senso de Negócios: A equipe da e

In [58]:
import pandas as pd
from pptx import Presentation
from pptx.util import Inches

In [ ]:

def add_comments_to_ppt(df, ppt_path, output_path):
    # Carregar a apresentação do PowerPoint
    prs = Presentation(ppt_path)
    
    # Verificar se o número de slides e o número de comentários correspondem
    if len(prs.slides) < len(df):
        raise ValueError("O número de slides no PowerPoint é menor que o número de linhas no DataFrame.")
    
    # Iterar sobre o DataFrame e adicionar comentários
    for idx, row in df.iterrows():
        slide = prs.slides[idx]
        comment = row['PPTV']

        slide.notes_slide.notes_text_frame.text = comment        
        # # Adicionar comentário ao slide
        # for shape in slide.shapes:
        #     if shape.has_text_frame:
        #         shape.text_frame.text = shape.text_frame.text + '\n\n' + comment

    # Salvar a apresentação modificada
    prs.save(output_path)

df = pd.DataFrame(df_score)

ppt_path = 'C:\\Users\\filip\\OneDrive\\Área de Trabalho\\Cast Group\\Projects\\for_all_see_it_cast\\data\\to_score\\Proposta Soluções Ágeis Cast - V3 14.11.24 1.pptx'
output_path = 'C:\\Users\\filip\\OneDrive\\Área de Trabalho\\Cast Group\\Projects\\for_all_see_it_cast\\data\\to_score\\Proposta Soluções Ágeis Cast PPTV - V3 14.11.24 1.pptx'
add_comments_to_ppt(df, ppt_path, output_path)


In [61]:
#!pip install msal requests

In [39]:
import subprocess

# Function to generate requirements.txt
def generate_requirements(output_file='requirements.txt'):
    with open(output_file, 'w') as f:
        subprocess.call(['pip', 'freeze'], stdout=f)

# Generate requirements.txt
generate_requirements()


FileNotFoundError: [WinError 2] The system cannot find the file specified

In [ ]:
# import os
# from pdf2image import convert_from_path
# from PIL import Image

# def pdf_to_images(pdf_path, output_dir):
#     # Certifique-se de que o diretório de saída existe
#     if not os.path.exists(output_dir):
#         os.makedirs(output_dir)
    
#     # Converter PDF para uma lista de imagens
#     pages = convert_from_path(pdf_path)
    
#     for page_num, page in enumerate(pages):
#         # Definir o caminho de saída para a imagem
#         output_path = os.path.join(output_dir, f"page_{page_num + 1}.png")
#         # Salvar a imagem
#         page.save(output_path, 'PNG')
#         print(f"Página {page_num + 1} salva como {output_path}")

# # Exemplo de uso
# pdf_path = 'path/to/your/file.pdf'
# output_dir = 'path/to/output/directory'
# pdf_to_images(pdf_path, output_dir)
